In [ ]:
import os
from google.cloud import speech, storage
from google.protobuf.json_format import MessageToDict
from pydub import AudioSegment
from kiwipiepy import Kiwi
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import librosa

In [ ]:
# Google Cloud 서비스 계정 키 파일 경로 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/test/gen-lang-client-0969588048-889a2e0ba98c.json"

In [ ]:
def convert_to_mono_and_resample(source_path, destination_path):
    """오디오 파일을 모노로 변환하고 샘플 속도를 48000 Hz로 변환"""
    audio = AudioSegment.from_wav(source_path)
    audio = audio.set_channels(1)  # 모노로 변환
    audio = audio.set_frame_rate(48000)  # 샘플 속도 변환
    audio.export(destination_path, format="wav")
    print(f"Converted {source_path} to mono and resampled to 48000 Hz as {destination_path}")

In [ ]:
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """파일을 Google Cloud Storage 버킷에 업로드"""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [ ]:
def transcribe_gcs(gcs_uri):
    """Google Cloud Storage에 저장된 오디오 파일을 텍스트로 변환"""
    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=48000,
        language_code="ko-KR",
        enable_word_time_offsets=True
    )
    operation = client.long_running_recognize(config=config, audio=audio)
    print("Waiting for operation to complete...")
    response = operation.result(timeout=10000)

    return response

In [ ]:
def process_audio_file(audio_file_path, bucket_name, gcs_filename, output_csv_path, output_audio_folder="segmented_audio"):
    # 모노 및 샘플 속도 변환
    mono_audio_file_path = audio_file_path
    convert_to_mono_and_resample(audio_file_path, mono_audio_file_path)

    # Google Cloud Storage에 오디오 파일 업로드
    upload_to_gcs(bucket_name, mono_audio_file_path, gcs_filename)
    gcs_uri = f"gs://{bucket_name}/{gcs_filename}"

    # GCS URI를 사용하여 음성 인식 수행
    response = transcribe_gcs(gcs_uri)
    print("음성을 텍스트로 변환 완료")

    # 변환된 텍스트 추출 및 타임스탬프 정보 가져오기
    words_info = []
    for result in response.results:
        alternative = result.alternatives[0]
        for word_info in alternative.words:
            word = word_info.word
            start_time = word_info.start_time.total_seconds()
            end_time = word_info.end_time.total_seconds()
            words_info.append((word, start_time, end_time))

    full_text = " ".join([word[0] for word in words_info])

    # Kiwi를 사용하여 문장 분리
    kiwi = Kiwi()
    sentences = kiwi.split_into_sents(full_text)

    # 출력 오디오 폴더가 없으면 생성
    if not os.path.exists(output_audio_folder):
        os.makedirs(output_audio_folder)

    output_data = []
    audio = AudioSegment.from_wav(mono_audio_file_path)
    current_index = 0

    for i, sentence in enumerate(sentences):
        sentence_text = sentence.text.strip()
        sentence_words = sentence_text.split()

        # 문장의 시작 시간과 끝 시간 계산
        start_time_sentence = None
        end_time_sentence = None

        for word_info in words_info[current_index:]:
            if word_info[0] == sentence_words[0] and start_time_sentence is None:
                start_time_sentence = word_info[1]
            if word_info[0] == sentence_words[-1]:
                end_time_sentence = word_info[2]
                break

        # 오디오 파일 분할
        start_ms = start_time_sentence * 1000  # 밀리초로 변환
        end_ms = end_time_sentence * 1000  # 밀리초로 변환
        segment = audio[start_ms:end_ms]

        # 분할된 오디오 파일 저장
        segment_filename = f"{output_audio_folder}/sentence_{i + 1}.wav"
        segment.export(segment_filename, format="wav")

        output_data.append([i + 1, segment_filename, sentence_text])
        print(f"문장 {i+1}: {sentence_text}, 파일 저장: {segment_filename}")

        current_index += len(sentence_words)

    # CSV 파일로 저장
    df = pd.DataFrame(output_data, columns=["ID", "PATH", "TEXT"])
    df.to_csv(output_csv_path, index=False, encoding='utf-8')

In [ ]:
# 오디오 파일 경로와 출력 CSV 경로 및 출력 오디오 폴더 지정
audio_file_path = '/content/drive/MyDrive/test/example.wav'  # 오디오 파일 경로
bucket_name = 'onto_bucket'  # Google Cloud Storage 버킷 이름
gcs_filename = 'audio_file.wav'
output_csv_path = '/content/drive/MyDrive/test/timestamps.csv'
output_audio_folder = '/content/drive/MyDrive/test/segmented_audio'

# 전체 프로세스 실행
process_audio_file(audio_file_path, bucket_name, gcs_filename, output_csv_path, output_audio_folder)

Converted /content/drive/MyDrive/test/example.wav to mono and resampled to 48000 Hz as /content/drive/MyDrive/test/example.wav
File /content/drive/MyDrive/test/example.wav uploaded to audio_file.wav.
Waiting for operation to complete...
음성을 텍스트로 변환 완료
문장 1: 오늘 토론을 주자는 공직선거에 의무투표제를 도입해야 한다 합니다, 파일 저장: /content/drive/MyDrive/test/segmented_audio/sentence_1.wav
문장 2: 먼저 양친의 기조연설 부터 시작하겠습니다, 파일 저장: /content/drive/MyDrive/test/segmented_audio/sentence_2.wav
문장 3: 시작하겠습니다 나타내는 표현입니다, 파일 저장: /content/drive/MyDrive/test/segmented_audio/sentence_3.wav
문장 4: 소재로 의무투표제 우리나라 실정의 부합하지 않기 때문입니다, 파일 저장: /content/drive/MyDrive/test/segmented_audio/sentence_4.wav
문장 5: 2016년 중앙선거관리위원회 딸이면 검색 강성우 투표제를 도입하고 있는 국가는 여덟 개 곡인데요 100만 명 정도로 인구수 같이 극히 적은 곳이 세 곳 군사독재의 의무투표제를 도입한 국가가 내 곳입니다, 파일 저장: /content/drive/MyDrive/test/segmented_audio/sentence_5.wav
문장 6: 인구수가 적지 않고 현재 민주주의의 속도가 높은 우리나라는 투표를 영어로 강제 할 필요가 없습니다, 파일 저장: /content/drive/MyDrive/test/segmented_audio/sentence_6.wav
문장 7: 나머지 한 곳인 호주는 직후 바로 음악 대표자

음성 감정 모델 불러오는 함수 및 전처리 함수

In [ ]:
def compute_melspectrogram_(audio_path, n_mels=128, hop_length=512, n_fft=2048):
    y, sr = librosa.load(audio_path)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length, n_fft=n_fft)

    return mel_spec

def preprocess_audio_(audio_path, target_length=256):
    mel_spec = compute_melspectrogram_(audio_path)
    current_length = mel_spec.shape[1]
    if current_length < target_length:
        mel_spec = np.pad(mel_spec, ((0, 0), (0, target_length - current_length)), mode='constant')
    elif current_length > target_length:
        start = (current_length - target_length) // 2
        mel_spec = mel_spec[:, start:start+target_length]

    return mel_spec

def load_data(csv_file_path):
    data = pd.read_csv(csv_file_path)
    X = []
    y = []

    for index, row in data.iterrows():
        audio_path = row['PATH']
        mel_spectrogram = preprocess_audio_(audio_path)
        X.append(mel_spectrogram)

    X = np.array(X)

    return X

In [ ]:
def process_predict(model_path, test_csv_path, output_csv_path):
    model = keras.models.load_model(model_path)
    X = load_data(test_csv_path)
    pred = model.predict(X)
    pred_classes = np.argmax(pred, axis=1)
    csv = pd.read_csv(test_csv_path)
    csv['pred'] = pred_classes
    csv.to_csv(output_csv_path, index=False)
    return csv

In [ ]:
t = pd.read_csv('/content/drive/MyDrive/test/timestamps.csv')
t

,ID,PATH,TEXT
0,1,/content/drive/MyDrive/test/segmented_audio/se...,오늘 토론을 주자는 공직선거에 의무투표제를 도입해야 한다 합니다
1,2,/content/drive/MyDrive/test/segmented_audio/se...,먼저 양친의 기조연설 부터 시작하겠습니다
2,3,/content/drive/MyDrive/test/segmented_audio/se...,시작하겠습니다 나타내는 표현입니다
3,4,/content/drive/MyDrive/test/segmented_audio/se...,소재로 의무투표제 우리나라 실정의 부합하지 않기 때문입니다
4,5,/content/drive/MyDrive/test/segmented_audio/se...,2016년 중앙선거관리위원회 딸이면 검색 강성우 투표제를 도입하고 있는 국가는 여덟...
5,6,/content/drive/MyDrive/test/segmented_audio/se...,인구수가 적지 않고 현재 민주주의의 속도가 높은 우리나라는 투표를 영어로 강제 할 ...
6,7,/content/drive/MyDrive/test/segmented_audio/se...,나머지 한 곳인 호주는 직후 바로 음악 대표자를 도입한 것으로 배정에 달라 우리나라...
7,8,/content/drive/MyDrive/test/segmented_audio/se...,둘째로 강제성 으로 인한 투표는 제대로 된 미니 반영을 받기 때문입니다
8,9,/content/drive/MyDrive/test/segmented_audio/se...,높은 투표율이 반드시 미니의 정확한 반영을 의미하지 않습니다
9,10,/content/drive/MyDrive/test/segmented_audio/se...,스스로 후보자를 알아보고 공약을 평가하는 것이 아닌 벌금을 회피하기 위한 행위는 큰...


실행 코드 부분

In [ ]:
model_path = '/content/drive/MyDrive/test/model_9.h5'
test_csv_path = '/content/drive/MyDrive/test/timestamps.csv'
output_csv_path = '/content/drive/MyDrive/test/result.csv'

result = process_predict(model_path, test_csv_path, output_csv_path)

1/1 [==============================] - 2s 2s/step


In [ ]:
result

,ID,PATH,TEXT,pred
0,1,/content/drive/MyDrive/test/segmented_audio/se...,오늘 토론을 주자는 공직선거에 의무투표제를 도입해야 한다 합니다,0
1,2,/content/drive/MyDrive/test/segmented_audio/se...,먼저 양친의 기조연설 부터 시작하겠습니다,0
2,3,/content/drive/MyDrive/test/segmented_audio/se...,시작하겠습니다 나타내는 표현입니다,0
3,4,/content/drive/MyDrive/test/segmented_audio/se...,소재로 의무투표제 우리나라 실정의 부합하지 않기 때문입니다,0
4,5,/content/drive/MyDrive/test/segmented_audio/se...,2016년 중앙선거관리위원회 딸이면 검색 강성우 투표제를 도입하고 있는 국가는 여덟...,0
5,6,/content/drive/MyDrive/test/segmented_audio/se...,인구수가 적지 않고 현재 민주주의의 속도가 높은 우리나라는 투표를 영어로 강제 할 ...,0
6,7,/content/drive/MyDrive/test/segmented_audio/se...,나머지 한 곳인 호주는 직후 바로 음악 대표자를 도입한 것으로 배정에 달라 우리나라...,0
7,8,/content/drive/MyDrive/test/segmented_audio/se...,둘째로 강제성 으로 인한 투표는 제대로 된 미니 반영을 받기 때문입니다,0
8,9,/content/drive/MyDrive/test/segmented_audio/se...,높은 투표율이 반드시 미니의 정확한 반영을 의미하지 않습니다,0
9,10,/content/drive/MyDrive/test/segmented_audio/se...,스스로 후보자를 알아보고 공약을 평가하는 것이 아닌 벌금을 회피하기 위한 행위는 큰...,0


In [ ]:
for index, row in df.iterrows():
    text = row['TEXT']
    prediction = row['pred']  # 예측 결과 열의 이름이 'prediction'인 경우

    # 예측 결과가 1인 경우 괄호 안에 넣습니다.
    if prediction == 1:
        print(f'({text})')
    else:
        print(text)

오늘 토론을 주자는 공직선거에 의무투표제를 도입해야 한다 합니다
먼저 양친의 기조연설 부터 시작하겠습니다
시작하겠습니다 나타내는 표현입니다
소재로 의무투표제 우리나라 실정의 부합하지 않기 때문입니다
2016년 중앙선거관리위원회 딸이면 검색 강성우 투표제를 도입하고 있는 국가는 여덟 개 곡인데요 100만 명 정도로 인구수 같이 극히 적은 곳이 세 곳 군사독재의 의무투표제를 도입한 국가가 내 곳입니다
(인구수가 적지 않고 현재 민주주의의 속도가 높은 우리나라는 투표를 영어로 강제 할 필요가 없습니다)
나머지 한 곳인 호주는 직후 바로 음악 대표자를 도입한 것으로 배정에 달라 우리나라의 그대로 적용할 수 없으며 조선일보는 호주의 높은 투표율이 정치교육과 투표시간연장 이라는 보완적 장치 를 도입한 그만하고 밝힌 바 있습니다
둘째로 강제성 으로 인한 투표는 제대로 된 미니 반영을 받기 때문입니다
높은 투표율이 반드시 미니의 정확한 반영을 의미하지 않습니다
스스로 후보자를 알아보고 공약을 평가하는 것이 아닌 벌금을 회피하기 위한 행위는 큰 의미가 없기 때문입니다
실제로 런던 대학교에서 할아버지 교수는 투표를 으로 방치할 경우 후보자가 누군지도 모르는 사람들이 선거에 참여 여부 투표 에지리 낮아질 수도 있다는 연구 결과를 발표한 바 있습니다
따라서 저희 뉴페이스팀 공직선거 의무투표제를 한다라는 토론논제 반대하는 파입니다
감사합니다
